In [1]:
#@title Preparing the data
# Load the libraries
import pandas as pd
pd.set_option('display.precision', 4)
import tensorflow as tf
# Machine Learning libraries
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# Load the data 
# [https://archive.ics.uci.edu/ml/machine-learning-databases/00294/CCPP.zip]
df = pd.read_excel('Data.xlsx')
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.25, 
                                                    random_state = 42)

# For Polynomial Regression
poly_reg = PolynomialFeatures(degree = 4)
X_train_poly = poly_reg.fit_transform(X_train)
X_test_poly = poly_reg.transform(X_test)

# For Support Vector Regression
sc_X = StandardScaler()
sc_y = StandardScaler()
X_train_sc = sc_X.fit_transform(X_train)
y_train_sc = sc_y.fit_transform(y_train.reshape(-1, 1))
X_test_sc = sc_X.transform(X_test)

# Save the R2 scores of models
scores = {} 

In [2]:
#@title Regression Models
def regressors():
  reg = {
      'Multiple Linear Regression' : LinearRegression(),
      'Polynomial Regression' : LinearRegression(),
      'Support Vector Regression' : SVR(kernel = 'rbf'),
      'Decision Tree Regression' : DecisionTreeRegressor(random_state = 42),
      'Random Forest Regression' : RandomForestRegressor(n_estimators = 10, 
                                                         random_state = 0),
  }
  return reg

# Training the Regression Models
models = regressors()
for name, model in models.items():
  if name == 'Polynomial Regression':
    model.fit(X_train_poly, y_train)
    y_pred = model.predict(X_test_poly)
    scores[name] = r2_score(y_test, y_pred)
  elif name == 'Support Vector Regression':
    model.fit(X_train_sc, y_train_sc.ravel())
    y_pred = sc_y.inverse_transform(model.predict(X_test_sc).reshape(-1, 1))
    scores[name] = r2_score(y_test, y_pred)
  else:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    scores[name] = r2_score(y_test, y_pred)

In [3]:
#@title Artificial Neural Network Model
def build_ann():
  """
  Builds a Artificial Neural Network as
  (I) Initializing a Sequential model,
  (II) 2 Hidden layers with 32 units,
  (II) Output layer with 1 output,
  (VI) Compiles the model with Adam optimizer 
       and mean_squared_error loss function.
  returns the built model
  """
  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.Dense(32, activation = 'relu'))
  model.add(tf.keras.layers.Dense(32, activation = 'relu'))
  model.add(tf.keras.layers.Dense(1)) # For regression, activation = None
  model.compile(optimizer = 'adam', loss = 'mean_squared_error')
  return model

# Training the ANN
ann = build_ann()
ann.fit(X_train, y_train, batch_size = 32, epochs = 100,
        validation_data = (X_test, y_test), verbose = 0)
y_pred = ann.predict(X_test)
scores['Artificial Neural Network'] = r2_score(y_test, y_pred)

In [4]:
#@title Results
result = pd.DataFrame(data = scores.values(),
                      columns = ['R-squared score'],
                      index = scores.keys())
result.style

,R-squared score
Multiple Linear Regression,0.9292
Polynomial Regression,0.9418
Support Vector Regression,0.9447
Decision Tree Regression,0.9301
Random Forest Regression,0.9582
Artificial Neural Network,0.9072
